# Air quality sensor documentation

## <u>Components</u>

## 1. Hardware
#### Composed by the SM300D2-V02 sensor, connected to a Raspberry Pi3B
<img src="./Images/RBSCon.png" width="500"/>


### 1.1. SM300D2-V02 Air Quality Sensor
-	It is a seven-in-one sensor module that collects and outputs CO2, formaldehyde, TVOC, PM2.5, PM10, temperature and humidity data.  

-	It adopts two output modes of RS485 bus and UART TTL level.  

-	The temperature is accurate to 0.1 ℃, and the humidity is accurate to 0.1% relative humidity.  

-	It outputs automatically the seven data parameters every second.   

-	Specifications:

**Name**|**Resolution**|**Range**|**Accuracy**
:-------|:------------:|:-------:|:----------:
eCO2||440ppm~5000ppm|±50ppm
eCH2O|1ug/m<sup>3</sup>|1ug~1000ug|
TVOC|1ug/m<sup>3</sup>|0ug~2000ug|
PM2.5|0.8ug/m<sup>3</sup>|5ug~1000ug|±10%
PM10||5ug~1000ug|±10%
Temperature|0.01ºC|-40ºC~125ºC|±0.5%ºC
Humidity|0.04%|0~100%|±3%RH

**Name**|**Specification**
:-------|:---------------:
Physical interface| ZH1.5-4
Output Data| UART-RS485
Operating Voltage|5.0±0.2VDC
Working current|≤60mA
Preheat time| 2 min
Operating temperature|0ºC~50ºC
Working humidity|≤95%RH
Dimensions|55.8x50x19.5mm (LxWxH)
Service life|5 years (in the air)

### 1.2. RaspberryPi3B

In our case we used an USB, instead of a SD card, as the memory of our RaspberryPi, as it is known to have a larger lifespan. If you aim to do the same, just download the Raspberry Pi operating system with the Raspberry Pi Imager in a USB.

### 1.3. RaspberryPi3B-Sensor connection
We connected the sensor with the RB through the rs485 connectors:  
  
  <img src="./Images/connections.png" width="500"/>  

Three wires are used to connect the RB and the sensor. In the image above, the upper part are the RBPi 3B connections and the bottom part shows the sensor connections. 
- The red wire connected to the sensor is the 5V power supply, which is connected to the 2nd RB pin (orange wire)  

- The black wire connected to the sensor sends data to the RB, this wire is connected to the 6th RB pin (yellow wire)  

- The green wire connected to the sensor is GND. It is connnected to the GND pin of the RB through the green wire  


The rs485 connector that is not used in this case receives data from the RB in other applications.

## 2. Software

### 2.1. On Raspberry Pi 3B
The software used as a python ide:

- **Thonny**: used as a programming environment, using python language. The scripts were used to:  
    -	Receive and read data from the sensor
    -	Send data to localhost and AWS (Amazon Web Services)
    -   Interchange data with Ubidots

### 2.2. On your PC

Raspberry remote desktop:
- VNC Viewer

Database visualization and manipulation:
- DBeaver (not strictly necessary)  

- VSCode

Data visualization (website):
- UBIDOTS

## <u>Steps - to do</u>

## 1. Visualization of the Raspberry Remote Desktop in your PC

### Connect RB Pi3B to VNC Viewer: RB Remote Desktop VNC
1. On your RB:

- To begin, open up the raspi-config tool by running the following command: sudo raspi-config
- Within the configuration tool, use the arrow keys to scroll down to "2. Display Options" and "D1. Resolution"
- Select the desired resolution. Usually DMT Mode 82 1920x1080 60Hz 16:9
- Press OK
- Within the configuration tool, use the arrow keys to scroll down to "3. Interface Options" and "P3. VNC"
- Select VNC enable
- Press OK and reboot

2. On your PC, where you want to visualize the desktop of your RB:

- First, you need to download VNC Viewer on your computer at https://www.realvnc.com/es/connect/download/viewer/raspberrypi/ 

- IP Adress:  
The connection to visualize the RB desktop in the VNC Viewer application, is made with IP adress of your RB. Therefore you must have Wi-Fi or ethernet connection on your RB to use VNC. 
To discover your IP adress, the easiest way is to connect the RB to a screen, mouse and keyboard.  
If you like, you can set a fixed IP address for ethernet and Wi-Fi connections, so you always know which IP to connect to, instead of checking it every time you need to connect.  
To fix the IP adress on your RB:  
  - Right-click on the network settings icon. It is the two arrows icon, on the top right corner of the screen.  
  - Select *Wireless & Wired Network Settings*. A new window pops up showing your network preferences.  
  - On the second drop-down select eth0. Fill the *IPv4 Address* with the desired IP address fot the ethernet connection.  
  - Click *Apply*  
  - On the second drop-down select wlan0. Fill the *IPv4 Address* with the desired IP address fot the Wi-Fi connection.  
  - Click *Apply*  
  - Close the window

Now that the RB IP address is fixed, open VNC Viewer on your computer, and introduce the fixed IP address on the top bar ("*Enter a VNC Server address or search*"). Keep in mind that the IP address you have to connect to, depends on the type of connection: if RB is connected through ethernet port to the network, insert the eth0 fixed IP address, if it is connected through Wi-Fi, use the wlan0 IP address. 

## 2. Previous RB downloads, installations or configuration settings

### Download Thonny ide for Python in the RB  
Open the RB terminal and run the lines:  
`sudo apt-get update`  
`sudo apt-get install python3-thonny`  

### Install mysql.connect module on RB  
Open the RB terminal and run the line:  
`sudo apt install python{,3}-mysql.connector`

### Make sure your Serial Port Interface is enabled in your RB (where the data from the sensor is coming from):
- Serial Port needs to be enabled or activated  
(Note: I found that when wanting to extract the sensor data using the python code, the permission was denied for opening the ttyS0 port.  
If you have the same problem, try disableling the Serial Console).



## 3. Code
There are four python scripts involved in the process that are run in the RB:  
  
CodiAdq.py, FuncionsSensor.py, FuncioMariaDB.py and FuncionsUbidots.py. 

### 3.1. CodiAdq.py
Main script. It calls out functions defined in the other three (FuncionsSensor.py, FuncionMariaDB.py and FuncionsUbidots.py) when it is executed.  

The main function is `sensor_reading()` that returns a dicctionary with the measures of the thread. The thread of `sensor_reading()` executed every 5 seconds.

#### 3.1.1. Run CodiAdq.py at boot: how to set the RB to execute the CodiAdq.py at its boot

_Leave this step for the end: it is recomended to be done after having done everything else when everything is working and ready_

It is an interesting feature because our RB-Sensor set is expected to be on top of an AGV robot. Of course, we are not going to be capable of constantly accessing and entering the RB to execute the code. It is therefore important that the created adquisition system executes constantly just by being connected to a current source.  
  
- Open the .bashrc script with the nano editor:  
`sudo nano /home/pi/.bashrc`

- Add the following lines at the end of the script:  
`echo Running at boot`  
`sudo python /home/pi/Desktop/Air_Sensor/CodiAdq.py`  
(if your CodiAdq.py has a different path del arxiu, you must change "/home..../" for it)

- "Crt + O" to save the changes  
  
  
- Reboot the RB to apply the configurated changes

### 3.2. FuncionsSensor.py
Contains the necessary functions to retrieve the data from the sensor.  
 
_Nothing needs to be changed in this script and no additional steps need to be followed._

### 3.3. FuncioMariaDB.py
Contains two functions that send the air quality obtained data to be stored to two MariaDB databases.  

A local MariaDB database and a Amazon Web Service MariaDB database:

In [ ]:
def localDB_function(sDB, dvar): #sDB is the name of the database and dvar is a dictionary with all the variables
        #mariadb connection
    db = mysql.connect(host="localhost", user="user",passwd="password",database=sDB) 
    cursor = db.cursor() 
        
        #insert information
    sql_insert = "INSERT INTO AirQuality (Time, CO2, Formaldehyde, TVOC, PM25, PM10, Temperature, Humidity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql_insert,(dvar['Temps'], dvar['CO2'], dvar['Formaldehil'], dvar['TVOC'],dvar['PM25'], dvar['PM10'], dvar['Temperatura'], dvar['Humitat'])) 
        
        #send to database
    db.commit()  
    cursor.close()
    db.close()

Where:  
  
host, user and password should be changed to yours.  
AirQuality is the name of the table  
*Time, CO2, Formaldehyde, TVOC, PM25, PM10, Temperature, Humidity* are the variables, which will be columns in DBeaver. 

In [ ]:
def awsDB_function(sawsDB, dvar):
    awsdb = mysql.connect(host="endpoint url", user="master username",passwd="master password",database=sawsDB)
    awsCursor = dbaws.cursor() 


            #insert information
    aws_insert = "INSERT INTO AWSMariaDB (Time, CO2, Formaldehyde, TVOC, PM25, PM10, Temperature, Humidity) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    awsCursor.execute(aws_insert,(dvar['Temps'], dvar['CO2'], dvar['Formaldehil'], dvar['TVOC'], dvar['PM25'], dvar['PM10'], dvar['Temperatura'], dvar['Humitat'])) 
            #it is sent to the database
    awsdb.commit()
    awsCursor.close()
    awsdb.close()

Where:  
 
host (endpoint), user (master username), passwd (master password), databse (initial database name).
AWSMariaDB is the name of the table you will have created in your AWS database.  
*Time, CO2, Formaldehyde, TVOC, PM25, PM10, Temperature, Humidity* are the columns (or variables) of your data.  
  
! Make sure the name of the created table (next step) corresponds to the name of the table in which you are inserting data with the awsDB_function()

#### 3.3.1. Creation of local and AWS MariaDB accounts and databases
In order to store the data to a table in these ddbb, it is firstly necessary to create them:  
  
  a) **Local MariaDB database: ** 
    
In our case, we used already existing MariaDB account with the following credentials:
  - user = "euss"
  - password = "rasppi"
  _(the creation of the database will be done at 3.3.2.)_

b) **Amazon Web Services: RDS: MariaDB database:  **

  1. Create an AWS account
  2. Create a database, following the instructions from: https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_CreateDBInstance.html  
  At step 6, choose MariaDB.  
    

  

  *Take note of your Endpoint, Master username, your Master password and the initial DB name, as they will be necessary in further steps*. 
      
      
Important:  

Beware that the free features of AWS are only free under certain conditions. Your free trial at AWS lasts up to 12 months. But besides this, there are certain tools that have a capacity limit which, if exeeded, will be charged (despite still being under your free trial).  
    In our case, our DB instance has a certain capacity limit. When the 85% of the free usage limit is exceeded, you will recieve an email (AWS Free Tier limit alert).  

To avoid exeeding the limit (and being charged), you need to terminate your running instances:  
1. Open the console at https://console.aws.amazon.com/ec2/  
2. Select "Instances" ("Instancias (en ejecución))"  
3. Select the running instances  
4. Go to Instance state ("Estado de la instancia"): Terminate instance ("Terminar")

#### 3.3.2. Visualization of the databases with DBeaver and VSCode
These two tools will be used for the:  

- Creation of the local database  

- Creation of a table in both local and AWS databases  

- VISUALIZATION of the data

Note: everything that is done with DBeaver can also be done with VSCode. It is therefore not necessary to have the two of them: you can chose one.

#### 3.3.2.1. DBeaver  
1. Download DBeaver  

2. Open DBeaver and select *New Connection*. 

3. Select MariaDB and click *Next*. 

4. Fill:
- Server Host: IP address (local) or Endpoint (AWS)
- Username: "euss" (local) or Master username (AWS) 
- Password: "rasppi" (local) or Master password (AWS). 

5. (Step only for the local database) In databases folder, create a new database.  

6. Inside the database, create a new table in the tables folder ("AirQuality", in this case)  

<img src="./Images/createt.png" width="300"/>. 

7. In the new table, create all the needed columns (variables).  

8. Also create a constraint defining an id as a primary key  
 
<img src="./Images/createt2.png" width="700"/>.  

Once the connection is well stablished, when running the CodiAdq.py script, the readings will appear in the table. 

#### 3.3.2.2. Visual Studio Code
To visualize your data at VSCode:  
- Download the _SQL Tools_ extension  

- A database icon will appear at the left vertical bar of your VSCode window: click there. 

- On connections, create a new connection:  
<img src="./Images/newconnection.png" width="300"/>. 

- Select MariaDB. 

- Fill the connection settings:
    Connection name: it is the name that you give to your connection in VSCode (up to you)
    Server Adress: url of the AWS db instance (or rb IP if you are connecting the local database)
    Port: 3306
    Database: (initial name of your database)
    Username: master username
    Use password: Save password (recommended)
    Password: master password. 

- Test your connection and save it. 

- Your connection should now appear under "CONNECTIONS"!

- If you have not done it with DBeaver, follow steps 5, 6, 7 and 8 of the previous instructions (3.3.2.1)  

### 3.3. FuncionsUbidots.py
To visualize the data nicely (plots, tables, etc) from any other device, the Ubidots platform was used. 
This permits sending values obtained from the sensor from the RB python script to Ubidots. At Ubidots, a user-friendly dashboard can be created to visualize this data.  
Additionally, it is also possible to retrieve data from ubidots to the python script. This permits creating and activating a kind of actuator.  
  
The FuncionsUbidots.py script contains the functions that send and retrieve, in other words, interchange data between our python scripts and ubidots.


#### 3.3.1. Ubidots Account
Go to https://industrial.ubidots.com/accounts/signin/ and sign up or log into your account.  
  
The account that we have been using has the following credentials:  
  
user: ubeussair  
mail: andreacostagutierrez@gmail.com 
password: UBair1234
  
  In order to interchange data, another important credential is your account TOKEN. It is the Ubidots account identifier that we will use in python scripts.  
go to the user dropdown button and go to API credentials.

#### 3.3.2. FuncionsUbidots.py and CodiAdq.py
It is important to change your TOKEN, DEVICE_LABEL and VARIABLE at CodiAdq.py if you are using another account.  

If you are using a brand new account, you do not have to necessarily create the device at ubidots before executing the code. If the device you have specified to send data to (DEVICE_LABEL) does not appear on your ubidots account, a device with that name will be automatically created when executing the code.  

Note that free accounts can have maxium of three devices: if you try to upload data to a fourth device, you will encounter an error.

#### 3.3.2.1 Sending data to Ubidots  
The python scripts send the measured data to the specified device (DEVICE_LABEL) of your Ubidots account (TOKEN), where it is kept in different variables. Below we can see an Ubidots example of the variables of the device named AirRubotMecanum:  

   <img src="./Images/variablesdevice.png" width="700"/>


#### 3.3.2.2. Obtaining data from Ubidots - Switch actuator
In order to have a control of the RB-Sensor module from any PC, a switch variable at Ubidots can be created at the dashboard:  
  
<img src="./Images/switch.png" width="300"/>.   <img src="./Images/switch2.png" width="300"/>
  
  Our switch variable takes the value 1 when it's on, and the value 0 when it's off.
    
  At FuncionsUbidots.py, the `get_var()` function, retrieves the value of a given variable of your Ubidots account. As it can be seen at CodiAdq.py, we use the function to obtain the value of this switch variable.  

We have stablished a condition in which the measuring thread (`sensor_reading()` function) is only executed if the switch variable takes a value of 1 (if the switch at ubidots is on).  
Therefore, we can control the data measurment through this button at the ubidots url.

#### 3.3.3. Sharing Ubidots URL

Once everything is ready, it is interesting to share the url to any user so that they can access the dashboard (not the account, just the dashboard).  
At Ubidots:
- Go to the Dashboards list drawer and select the "share" icon 
- Click the "copy" icon to copy the dashboard's public URL to the clipboard
- Anywone who pastes the copied public URL into a web browser tab will be taken to our dashboard  
  
  For example, our interface (where the data can be visualized and the air sensor activity can be controlled) is at https://stem.ubidots.com/app/dashboards/public/dashboard/bfSkjwxrNwNfR5va6jaEctTSrvuV4ZK-XjLaqv9-7Oc 